# Lightning Channel via Batch Transaction

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup LND GRPC

In [ ]:
import sys

In [ ]:
# LND GRPC bindings should be in '/home/jovyan/lnrpc'
sys.path.append('/home/jovyan/lnrpc')

In [ ]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [ ]:
from lnd_rpc import LndRpc, lnmsg, walletmsg, signrpc, signmsg, routermsg

In [ ]:
ali = LndRpc(host='ali-lnd:10009',
             cert_path='/ali-lnd/tls.cert',
             macaroon_path='/tmp/ali.macaroon')

In [ ]:
bob = LndRpc(host='bob-lnd:10009',
             cert_path='/bob-lnd/tls.cert',
             macaroon_path='/tmp/bob.macaroon')

In [ ]:
aliNodePubKey = ali.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
aliNodePubKey

In [ ]:
bobNodePubKey = bob.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
bobNodePubKey

## Fund Ali and Bob

In [ ]:
import os
import bitcoin
from rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

In [ ]:
brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [ ]:
brpc.getblockcount()

In [ ]:
aliAddr = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
aliAddr

In [ ]:
bobAddr = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
bobAddr

In [ ]:
# fund ali
_ = list(brpc.generatetoaddress(1, aliAddr.address))

In [ ]:
# fund bob
_ = list(brpc.generatetoaddress(1, bobAddr.address))

In [ ]:
brpc.createwallet('miner')
minerAddr = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

In [ ]:
# unlock mined coins
_ = list(brpc.generatetoaddress(110, minerAddr))

In [ ]:
ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

In [ ]:
bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

## Create bid and ask

In [ ]:
import base64
import bitcoin.core as bc
import bitcoin.core.script as bs
import bitcoin.wallet as bw
from bitcoin.rpc import hexlify, unhexlify

### Ali taker

In [ ]:
# taker needs to pay premium to open channel
premiumAmount = int(0.001 * bc.COIN) # premium Ali is willing to pay
fundAmount = int(0.16 * bc.COIN) # requested inbound capacity

In [ ]:
# create dummy psbt to extract 'funding' UTXO and change addresses
# user minerAddr as a dummy address to create psbt
tx_template = walletmsg.TxTemplate(outputs={str(minerAddr): premiumAmount})

psbtRequest = walletmsg.FundPsbtRequest(raw=tx_template, target_conf=6)

In [ ]:
aliDummyPsbt = ali.wallet.FundPsbt(request=psbtRequest)
aliDummyPsbt

In [ ]:
psbtBase64 = base64.b64encode(aliDummyPsbt.funded_psbt).decode()

In [ ]:
brpc._proxy._call('analyzepsbt', psbtBase64)

In [ ]:
aliDummyPsbtDecoded = brpc._proxy._call('decodepsbt', psbtBase64)
aliDummyPsbtDecoded

In [ ]:
aliInputs = []
for vin in aliDummyPsbtDecoded['tx']['vin']:
    txin = bc.CTxIn(bc.COutPoint(bc.lx(vin['txid']), vin['vout']))
    aliInputs.append(txin)
aliInputs

In [ ]:
# assume there are only 2 outputs: change and channel funding
assert len(aliDummyPsbtDecoded['tx']['vout']) == 2

In [ ]:
aliChangeOutput = aliDummyPsbtDecoded['tx']['vout'][aliDummyPsbt.change_output_index]
aliChangeOutput = bc.CTxOut(int(aliChangeOutput['value'] * bc.COIN),
                            bc.CScript(unhexlify(aliChangeOutput['scriptPubKey']['hex'])))

aliChangeOutput

In [ ]:
# Generate pubkey for channel funding
key_family = 0 # multisig?
keyReq = walletmsg.KeyReq(key_family=key_family)
aliKeyDesc = ali.wallet.DeriveNextKey(keyReq)
aliKeyDesc = ali.wallet.DeriveNextKey(keyReq) # need to call twice?
aliKeyDesc

In [ ]:
takeRequest = {
    'nodeAddr': 'ali-lnd',
    'nodePubKey': aliNodePubKey,
    'premiumAmount': premiumAmount,
    'fundAmount': fundAmount,
    'inputs': aliInputs,
    'change': aliChangeOutput,
    'multisigKey': aliKeyDesc # only need to send bytes of pub key and not key locator
}
takeRequest

### Bob maker

In [ ]:
premiumAmount = int(0.001 * bc.COIN) # premium Bob is willing to accept
fundAmount = int(0.16 * bc.COIN) # funding amount Bob is willing to provide

In [ ]:
# create dummy psbt to extract 'funding' UTXO and change addresses
# user minerAddr as a dummy address to create psbt
tx_template = walletmsg.TxTemplate(outputs={str(minerAddr): fundAmount})

psbtRequest = walletmsg.FundPsbtRequest(raw=tx_template, target_conf=6)
psbtRequest

In [ ]:
bobDummyPsbt = bob.wallet.FundPsbt(request=psbtRequest)
bobDummyPsbt

In [ ]:
psbtBase64 = base64.b64encode(bobDummyPsbt.funded_psbt).decode()

In [ ]:
bobDummyPsbtDecoded = brpc._proxy._call('decodepsbt', psbtBase64)
bobDummyPsbtDecoded

In [ ]:
bobInputs = []
for vin in bobDummyPsbtDecoded['tx']['vin']:
    txin = bc.CTxIn(bc.COutPoint(bc.lx(vin['txid']), vin['vout']))
    bobInputs.append(txin)
bobInputs

In [ ]:
# assume there are only 2 outputs: change and channel funding
assert len(bobDummyPsbtDecoded['tx']['vout']) == 2

In [ ]:
bobChangeOutput = bobDummyPsbtDecoded['tx']['vout'][bobDummyPsbt.change_output_index]
bobChangeOutput = bc.CTxOut(int(bobChangeOutput['value'] * bc.COIN),
                            bc.CScript(unhexlify(bobChangeOutput['scriptPubKey']['hex'])))

bobChangeOutput

In [ ]:
key_family = 0 # multisig?
keyReq = walletmsg.KeyReq(key_family=key_family)
bobKeyDesc = bob.wallet.DeriveNextKey(keyReq)
bobKeyDesc = bob.wallet.DeriveNextKey(keyReq) # need to call twice?
bobKeyDesc

In [ ]:
makeRequest = {
    'nodeAddr': 'bob-lnd',
    'nodePubKey': bobNodePubKey,
    'premiumAmount': premiumAmount,
    'fundAmount': fundAmount,
    'inputs': bobInputs,
    'change': bobChangeOutput,
    'multisigKey': bobKeyDesc # only need to send bytes of pub key and not key locator
}
makeRequest

## Auctioneer

In [ ]:
import hashlib

### create batch funding transaction

In [ ]:
takerPubKey = takeRequest['multisigKey'].raw_key_bytes
makerPubKey = makeRequest['multisigKey'].raw_key_bytes

In [ ]:
# https://github.com/lightningnetwork/lightning-rfc/blob/master/03-transactions.md#funding-transaction-output
# https://github.com/bitcoin/bips/blob/master/bip-0069.mediawiki
# if makerPubKey > takerPubKey:
if list(makerPubKey) > list(takerPubKey):
    pk1, pk2 = takerPubKey, makerPubKey
else:
    pk1, pk2 = makerPubKey, takerPubKey

In [ ]:
msigScript = bs.CScript([
        bs.OP_2,
        pk1,
        pk2,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])

In [ ]:
# convert to P2WSH
scriptPubKey = bc.CScript([bs.OP_0, hashlib.sha256(msigScript).digest()])
scriptPubKey

In [ ]:
assert scriptPubKey.is_witness_v0_scripthash()

In [ ]:
fundingOutput = bc.CTxOut(takeRequest['premiumAmount'] + makeRequest['fundAmount'],
                          scriptPubKey)
fundingOutput

In [ ]:
batchInputs = takeRequest['inputs'] + makeRequest['inputs']
batchOutputs = [fundingOutput, takeRequest['change'], makeRequest['change']]
fundingOutputIdx = 0

In [ ]:
batchTx = bc.CTransaction(batchInputs, batchOutputs)
batchTx

In [ ]:
batchTxId = batchTx.GetTxid()
batchTxId

### Response to matched taker and maker

In [ ]:
import time

In [ ]:
# something that is unique to the corresponding matched taker and maker
channelId = hashlib.sha256((bobNodePubKey + aliNodePubKey + str(time.time())).encode()).hexdigest()

In [ ]:
takeResponse = {
    'batchTx': batchTx,
    'fundingOutputIdx': fundingOutputIdx,
    'channelId': channelId,
    'maker': {
        'nodeAddr': makeRequest['nodeAddr'],
        'nodePubKey': makeRequest['nodePubKey'],
        'premiumAmount': makeRequest['premiumAmount'],
        'fundAmount': makeRequest['fundAmount'],
        'remoteKey': makeRequest['multisigKey'].raw_key_bytes
    }
}

In [ ]:
makeResponse = {
    'batchTx': batchTx,
    'fundingOutputIdx': fundingOutputIdx,
    'channelId': channelId,
    'taker': {
        'nodeAddr': takeRequest['nodeAddr'],
        'nodePubKey': takeRequest['nodePubKey'],
        'premiumAmount': takeRequest['premiumAmount'],
        'fundAmount': takeRequest['fundAmount'],
        'remoteKey': takeRequest['multisigKey'].raw_key_bytes
    }
}

## Ali taker

In [ ]:
takeResponse

### Register funding shim to get ready for funding
this will also prompt taker to connect to maker

In [ ]:
maker = takeResponse['maker']
batchTx = takeResponse['batchTx']

In [ ]:
chan_point = lnmsg.ChannelPoint(funding_txid_bytes=batchTx.GetTxid(),
                                output_index=takeResponse['fundingOutputIdx'])

In [ ]:
local_key = lnmsg.KeyDescriptor(
    raw_key_bytes=aliKeyDesc.raw_key_bytes,
    key_loc=lnmsg.KeyLocator(
        key_family=aliKeyDesc.key_loc.key_family,
        key_index=aliKeyDesc.key_loc.key_index))

In [ ]:
chan_point_shim = lnmsg.ChanPointShim(
    amt=maker['fundAmount'] + maker['premiumAmount'],
    chan_point=chan_point,
    local_key=local_key,
    remote_key=maker['remoteKey'],
    pending_chan_id=unhexlify(takeResponse['channelId']),
    thaw_height=0 # set 0 for simplicity
)

In [ ]:
shim = lnmsg.FundingShim(chan_point_shim=chan_point_shim)

In [ ]:
# empty response
ali.lnd.FundingStateStep(ftm)

### Connect to maker

In [ ]:
addr = lnmsg.LightningAddress(
    pubkey=maker['nodePubKey'],
    host=maker['nodeAddr']
)

In [ ]:
connect_peer_req = lnmsg.ConnectPeerRequest(
    addr=addr
)

In [ ]:
ali.lnd.ConnectPeer(connect_peer_req)

## Bob maker

In [ ]:
makeResponse

### Initiate open channel request

In [ ]:
taker = makeResponse['taker']
batchTx = makeResponse['batchTx']

In [ ]:
chan_point = lnmsg.ChannelPoint(funding_txid_bytes=batchTx.GetTxid(),
                                output_index=takeResponse['fundingOutputIdx'])

In [ ]:
chan_point = lnmsg.ChannelPoint(
    funding_txid_bytes=batchTx.GetTxid(),
    output_index=makeResponse['fundingOutputIdx']
)

In [ ]:
local_key = lnmsg.KeyDescriptor(
    raw_key_bytes=bobKeyDesc.raw_key_bytes,
    key_loc=lnmsg.KeyLocator(
        key_family=bobKeyDesc.key_loc.key_family,
        key_index=bobKeyDesc.key_loc.key_index
    )
)

In [ ]:
chan_point_shim = lnmsg.ChanPointShim(
    amt=taker['fundAmount'] + taker['premiumAmount'],
    chan_point=chan_point,
    local_key=local_key,
    remote_key=taker['remoteKey'],
    pending_chan_id=unhexlify(makeResponse['channelId']),
    thaw_height=0 # for simplicity
)

In [ ]:
funding_shim = lnmsg.FundingShim(chan_point_shim=chan_point_shim)

In [ ]:
open_chan_req = lnmsg.OpenChannelRequest(
    node_pubkey=unhexlify(taker['nodePubKey']),
    local_funding_amount=taker['fundAmount'] + taker['premiumAmount'],
    push_sat=taker['premiumAmount'], # premium is pushed to the remote end
    funding_shim=funding_shim
)

In [ ]:
chanEventStream = bob.lnd.OpenChannel(open_chan_req)

In [ ]:
next(chanEventStream)

In [ ]:
# at this point Ali and Bob should have a pending channel
bob.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

In [ ]:
ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

## Sign batch transaction

### Ali taker

In [ ]:
takeResponse['batchTx']

#### Option 1: Using SignPsbt

In [ ]:
pbst = brpc._proxy._call('converttopsbt', hexlify(takeResponse['batchTx'].serialize()))
pbst

In [ ]:
psbtRequest = walletmsg.SignPsbtRequest(psbt=base64.b64decode(pbst))

aliSignedPsbt = ali.wallet.SignPsbt(psbtRequest)
aliSignedPsbt

In [ ]:
psbtDecoded = brpc._proxy._call('decodepsbt', base64.b64encode(aliSignedPsbt.signed_psbt).decode())
psbtDecoded

In [ ]:
takerUpdate = {
    'signed_psbt': base64.b64encode(aliSignedPsbt.signed_psbt).decode()
}

#### Option 2: Using SignOutputRaw

In [ ]:
# for simplicity assume single input
assert len(aliDummyPsbtDecoded['inputs']) == 1

In [ ]:
# for simplicity assume single signing key
assert len(aliDummyPsbtDecoded['inputs'][0]['bip32_derivs']) == 1

In [ ]:
input_ = aliDummyPsbtDecoded['inputs'][0]
input_

In [ ]:
aliP2wkh = bw.P2WPKHBitcoinAddress(aliAddr.address)
aliP2wkh

In [ ]:
pubkey = unhexlify(input_['bip32_derivs'][0]['pubkey'])
pubkey

In [ ]:
sign_desc = signmsg.SignDescriptor(
    key_desc=signmsg.KeyDescriptor(
        raw_key_bytes=pubkey
    ),
    witness_script=aliP2wkh.to_redeemScript(),
    output=signmsg.TxOut(
        value=int(input_['witness_utxo']['amount'] * bc.COIN),
        pk_script=aliP2wkh.to_scriptPubKey()
    ),
    sighash=bs.SIGHASH_ALL,
    input_index=0 # for simplicity taker's input is 0
)

In [ ]:
signReq = signmsg.SignReq(
    raw_tx_bytes=takeResponse['batchTx'].serialize(),
    sign_descs=[sign_desc]
)

In [ ]:
aliSignResp = ali.signer.SignOutputRaw(signReq)
aliSignResp

In [ ]:
signature = aliSignResp.raw_sigs[0] + bytes([bs.SIGHASH_ALL])

In [ ]:
witness = [signature, pubkey]
bs.CScriptWitness(witness)

In [ ]:
aliSignedTx = bc.CTransaction(
    vin=takeResponse['batchTx'].vin,
    vout=takeResponse['batchTx'].vout,
    witness=bc.CTxWitness([bc.CTxInWitness(bs.CScriptWitness(witness))]))

In [ ]:
# transactions with signatures cannot be converted to psbt
# pass the partially signed transaction as is. In practice we can pass it as bytes
takerUpdate = {
    'signed_tx': aliSignedTx
}

### Bob maker

In [ ]:
makeResponse['batchTx']

### Option 1: Using SignPsbt

In [ ]:
pbst = brpc._proxy._call('converttopsbt', hexlify(makeResponse['batchTx'].serialize()))
pbst

In [ ]:
psbtRequest = walletmsg.SignPsbtRequest(psbt=base64.b64decode(pbst))

bobSignedPsbt = bob.wallet.SignPsbt(psbtRequest)
bobSignedPsbt

In [ ]:
psbtDecoded = brpc._proxy._call('decodepsbt', base64.b64encode(bobSignedPsbt.signed_psbt).decode())
psbtDecoded

In [ ]:
makerUpdate = {
    'signed_psbt': base64.b64encode(bobSignedPsbt.signed_psbt).decode()
}

### Option 2: Using SignOutputRaw

In [ ]:
# for simplicity assume single input
assert len(bobDummyPsbtDecoded['inputs']) == 1

In [ ]:
# for simplicity assume single signing key
assert len(bobDummyPsbtDecoded['inputs'][0]['bip32_derivs']) == 1

In [ ]:
input_ = bobDummyPsbtDecoded['inputs'][0]
input_

In [ ]:
bobP2wkh = bw.P2WPKHBitcoinAddress(bobAddr.address)
bobP2wkh

In [ ]:
pubkey = unhexlify(input_['bip32_derivs'][0]['pubkey'])

In [ ]:
sign_desc = signmsg.SignDescriptor(
    key_desc=signmsg.KeyDescriptor(
        raw_key_bytes=pubkey
    ),
    witness_script=bobP2wkh.to_redeemScript(),
    output=signmsg.TxOut(
        value=int(input_['witness_utxo']['amount'] * bc.COIN),
        pk_script=bobP2wkh.to_scriptPubKey()
    ),
    sighash=bs.SIGHASH_ALL,
    input_index=1 # for simplicity maker's input is 1
)

In [ ]:
signReq = signmsg.SignReq(
    raw_tx_bytes=makeResponse['batchTx'].serialize(),
    sign_descs=[sign_desc]
)

In [ ]:
bobSignResp = bob.signer.SignOutputRaw(signReq)
bobSignResp

In [ ]:
signature = bobSignResp.raw_sigs[0] + bytes([bs.SIGHASH_ALL])

In [ ]:
witness = [signature, pubkey]
bs.CScriptWitness(witness)

In [ ]:
bobSignedTx = bc.CTransaction(
    vin=makeResponse['batchTx'].vin,
    vout=makeResponse['batchTx'].vout,
    witness=bc.CTxWitness([bc.CTxInWitness(bs.CScriptWitness(witness))]))
bobSignedTx

In [ ]:
# transactions with signatures cannot be converted to psbt
# pass the partially signed transaction as is. In practice we can pass it as bytes
makerUpdate = {
    'signed_tx': bobSignedTx
}

## Auctioneer. Finalize and publish batch transaction

### Option 1. combine, finalize and submit psbt

In [ ]:
takerUpdate

In [ ]:
makerUpdate

In [ ]:
combinedPsbt = brpc._proxy._call('combinepsbt', [takerUpdate['signed_psbt'], makerUpdate['signed_psbt']])
combinedPsbt

In [ ]:
brpc._proxy._call('decodepsbt', combinedPsbt)

In [ ]:
res = brpc._proxy._call('finalizepsbt', combinedPsbt)
res

In [ ]:
signedBatchTx = bc.CTransaction.deserialize(unhexlify(res['hex']))
signedBatchTx

### Option 2. Manually combine partially signed transactions

In [ ]:
takerUpdate

In [ ]:
makerUpdate

In [ ]:
# for simplicity assume we are only spending witness outputs
signedBatchTx = bc.CTransaction(
    vin=batchTx.vin,
    vout=batchTx.vout,
    witness=bc.CTxWitness([
        takerUpdate['signed_tx'].wit.vtxinwit[0],
        makerUpdate['signed_tx'].wit.vtxinwit[0]])) # [taker, maker]
signedBatchTx

### Submit batch tx

In [ ]:
batchTxId2 = brpc.sendrawtransaction(signedBatchTx)
batchTxId2

In [ ]:
assert batchTxId == batchTxId2

In [ ]:
_ = list(brpc.generatetoaddress(6, minerAddr))

In [ ]:
brpc.getrawtransaction(batchTxId2, verbose=True)

## Check that channel is open

In [ ]:
# should match the channel point
bc.b2lx(batchTxId2)

In [ ]:
# should be no pending channels
ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

In [ ]:
ali.lnd.ListChannels(lnmsg.ListChannelsRequest())

## Send a payment Bob to Ali

### Ali creates invoice

In [ ]:
invoice = lnmsg.Invoice(
    memo="thank you for using the auction",
    value=100
)


In [ ]:
invoiceResp = ali.lnd.AddInvoice(invoice)
invoiceResp

### Bob pays invoice

In [ ]:
# check invoice
paymentRequest = bob.lnd.DecodePayReq(
    lnmsg.PayReqString(pay_req=invoiceResp.payment_request)
)
paymentRequest

In [ ]:
sendPaymentResponse = bob.router.SendPaymentV2(
    routermsg.SendPaymentRequest(
        payment_request=invoiceResp.payment_request,
        timeout_seconds=60
))
sendPaymentResponse

In [ ]:
next(sendPaymentResponse)

### Check channel balance

In [ ]:
ali.lnd.ListChannels(lnmsg.ListChannelsRequest())